In [22]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from functools import reduce

In [26]:
dataset_2016 = 'Burlington_StatsCanada_2016_Census.csv'
dataset_2021 = 'Burlington_StatsCanada_2021_Census.csv'
private_schools = 'Private School Boards Ontario.csv'
public_schools = 'Public School Boards Ontario.csv'
house_sale_median = 'Burlington_Median_Sale_Price.csv'
house_rent_median = 'Burlington_Median_Rental_Price.csv'
house_sale_average = 'Burlington_Avg_Sale_Price.csv'
house_rent_average = 'Burlington_Avg_Rental_Price.csv'
df_2016 = pd.read_csv(dataset_2016, encoding = "ISO-8859-1", on_bad_lines = 'skip', low_memory = True, engine = 'python')
df_2021 = pd.read_csv(dataset_2021, encoding = "ISO-8859-1", on_bad_lines = 'skip', low_memory = True, engine = 'python')
df_private_schools = pd.read_csv(private_schools, encoding = "ISO-8859-1", on_bad_lines = 'skip', low_memory = True, engine = 'python')
df_public_schools = pd.read_csv(public_schools, encoding = "ISO-8859-1", on_bad_lines = 'skip', low_memory = True, engine = 'python')
df_sale_median = pd.read_csv(house_sale_median, encoding = "ISO-8859-1", on_bad_lines = 'skip', low_memory = True, engine = 'python')
df_rent_median = pd.read_csv(house_rent_median, encoding = "ISO-8859-1", on_bad_lines = 'skip', low_memory = True, engine = 'python')
df_sale_average = pd.read_csv(house_sale_average, encoding = "ISO-8859-1", on_bad_lines = 'skip', low_memory = True, engine = 'python')
df_rent_average = pd.read_csv(house_rent_average, encoding = "ISO-8859-1", on_bad_lines = 'skip', low_memory = True, engine = 'python')
df_public_schools.sample(3)

,ï»¿Board Number,Board Name,Board Type,School Number,School Name,School Type,School Special Condition Code,School Level,School Language,Grade Range,...,Change in Grade 6 Writing Achievement Over Three Years,Percentage of Grade 6 Students Achieving the Provincial Standard in Mathematics,Change in Grade 6 Mathematics Achievement Over Three Years,Percentage of Grade 9 Students Achieving the Provincial Standard in Mathematics,Change in Grade 9 Mathematics Achievement Over Three Years,Percentage of Students That Passed the Grade 10 OSSLT on Their First Attempt,Change in Grade 10 OSSLT Literacy Achievement Over Three Years,Percentage of School-Aged Children Who Live in Low-Income Households,"Percentage of Students Whose Parents Have No Degree, Diploma or Certificate",Extract Date
2453,B66184,Ottawa-Carleton DSB,Pub Dist Sch Brd (E/F),480428,Regina Street Alternative School,Public,Not applicable,Elementary,English,JK-6,...,NaN,43%,NaN,NaN,NaN,NaN,NaN,30,10,3-Nov-23
3769,B66052,Toronto DSB,Pub Dist Sch Brd (E/F),216402,Glamorgan Junior Public School,Public,Not applicable,Elementary,English,JK-6,...,NaN,28%,NaN,NaN,NaN,NaN,NaN,40,10,3-Nov-23
922,B67083,Dufferin-Peel CDSB,Cath Dist Sch Brd (E/F),708038,Georges Vanier Catholic School,Catholic,Not applicable,Elementary,English,JK-8,...,NaN,34%,NaN,NaN,NaN,NaN,NaN,20,10,3-Nov-23


In [28]:
housing_dfs = [df_sale_median, df_rent_median, df_sale_average, df_rent_average]
df_housing_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Primary Year'], how='outer'), housing_dfs)
df_housing_merged.head()

,Primary Year,"Sale Price, Median","Original Price, Median","Sale Price, Average","Original Price, Average"
0,2016,"$609,000","$2,400","$663,675","$2,710"
1,2017,"$690,000","$2,400","$763,549","$2,779"
2,2018,"$660,000","$2,400","$735,275","$2,711"
3,2019,"$690,000","$2,500","$755,017","$2,636"
4,2020,"$800,000","$2,500","$877,311","$2,683"


In [38]:
private_school_count = len(df_private_schools.loc[df_private_schools['City'] == 'Burlington'])

print(df)

2


In [50]:
public_school_clean = df_public_schools.loc[df_public_schools['City'] == 'Burlington']['School Type']
public_school_count = public_school_clean.value_counts()
public_school_count

School Type
Public      38
Catholic    17
Name: count, dtype: int64